Imports essential libraries (pandas, numpy, matplotlib, sklearn, scipy, os). Configures matplotlib for plotting and sets global warnings to be ignored. Defines the standard file path for data (features.parquet) and a directory for saving diagnostic results (results/q7_diagnostics).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
# from sklearn.metrics import r2_score # 不再使用
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
import os

# 基础配置
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

# 数据/结果路径
DATA_PATH = "features.parquet"
RESULTS_DIR = "results/q7_diagnostics"
os.makedirs(RESULTS_DIR, exist_ok=True)

This function defines the specific Out-of-Sample R-squared ($R^2_{OOS}$) metric required for financial forecasting, based on the methodology of Gu, Kelly, & Xiu (2020). The key feature of this $R^2$ is that the denominator (the measure of total variance, $SST$) is calculated as the sum of squared actual returns ($\sum y_{true}^2$), rather than the standard sum of squared deviations from the mean. This metric is used to evaluate the economic significance of the model's predictions.

In [2]:
def r2_oos_gkx(y_true, y_pred):
    """
    计算 Gu-Kelly-Xiu (2020) 要求的 OOS R²。
    分母是 y_true 的平方和（相对于0），而不是相对于均值。
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    if y_true.shape[0] == 0:
        return np.nan
    
    sse = np.sum((y_true - y_pred)**2)
    sst = np.sum(y_true**2) # GKX 关键点：分母是 y^2 的和
    
    if sst == 0: 
        return np.nan
        
    return 1.0 - (sse / sst)

This function is responsible for robustly loading and cleaning the financial data.

It reads the features.parquet file and identifies the feature and target columns.

Target Filtering: It filters out any rows where the target variable (ret_excess_t_plus_1) is missing.

Two-Stage Feature Imputation (Crucial Fix):

Stage 1 (Cross-Sectional Median): It performs the standard practice of monthly cross-sectional median imputation, replacing missing feature values in a given month with that month's median for that feature.

Stage 2 (Fallback to Zero): It applies a fallback: any remaining missing values (which only occur if a feature is entirely missing for an entire month, preventing a median calculation) are filled with 0. This fix ensures that the early period data (1957-1974), which contained these "all-NaN" months, is retained for training.

It returns the cleaned data, features, and target variables.

In [11]:
def load_data():
    """加载数据（修正了插值逻辑以处理全NaN月份）"""
    print("加载数据...")
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"数据文件未找到: {DATA_PATH}。请确保 features.parquet 在正确路径。")
    
    df = pd.read_parquet(DATA_PATH)
    
    if 'month' not in df.columns:
        raise KeyError("数据中未找到 'month' 列，无法继续。")
    df['month'] = pd.to_datetime(df['month'])
    
    feature_cols = [col for col in df.columns if col.startswith(('c_', 'm_', 'sic_'))]
    if 'c_mvel1' not in feature_cols:
         warnings.warn("警告：未找到市值特征 'c_mvel1'。规模分组分析将失败。")
         
    print(f"特征列数: {len(feature_cols)}")
    
    y = df['ret_excess_t_plus_1'].astype(np.float32)
    X = df[feature_cols].astype(np.float32)
    
    # 1. 丢弃没有目标(y)的样本
    y_mask = ~y.isnull()
    X, y, df = X[y_mask], y[y_mask], df[y_mask]
    
    # 2. 按月份对特征 X 进行中位数插值
    df['month_period'] = df['month'].dt.to_period("M")
    print("...开始月度中位数插值...")
    X_imputed = X.groupby(df['month_period']).transform(lambda x: x.fillna(x.median()))
    print("...月度中位数插值完成。")
    
    # ### 关键修正 ###
    # 如果一个月的所有值都为NaN，则该月的中位数也为NaN，插值失败。
    # 我们在这里添加一个“后备”插值，用 0 填充那些仍然为 NaN 的值。
    print("...开始后备插值 (fillna(0))...")
    X_imputed_fallback = X_imputed.fillna(0)
    print("...后备插值完成。")

    # 3. 丢弃插值后 X 中仍有缺失值的样本 (现在不应该有)
    x_mask = ~X_imputed_fallback.isnull().any(axis=1)
    X, y, df = X_imputed_fallback[x_mask], y[x_mask], df[x_mask]
    
    if len(df) == 0:
        raise ValueError("数据加载和清洗后，剩余样本为0。请检查数据源或清洗逻辑。")
        
    print(f"数据加载完成，最终有效样本数: {len(df)}")
    return df, X, y, feature_cols

This function trains the baseline Elastic Net model and evaluates its initial performance.Data Splitting: It splits the data into three chronological segments: Training (1957–1974), Validation (1975–1986), and Test (1987–2016).

Standardization: It fits a StandardScaler only on the training set and uses it to transform all three sets.

Model Training: It trains the baseline Elastic Net model with fixed hyperparameters ($\alpha=0.5$ and $l1\_ratio=0.5$).Evaluation: 

It calculates the  Roos² (using the r2_oos_gkx metric) for both the validation and test sets and prints the results.

In [4]:
def train_elastic_net(df, X, y):
    """训练弹性网络模型（作业基准模型），用于后续诊断"""
    print("\n训练弹性网络模型（基准参数：alpha=0.5, l1_ratio=0.5）...")
    
    print(f"数据时间范围: {df['month'].min().date()} 到 {df['month'].max().date()}")
    print(f"训练集时间范围: 1957-01-01 到 1974-12-31")
    print(f"验证集时间范围: 1975-01-01 到 1986-12-31")
    print(f"测试集时间范围: 1987-01-01 到 2016-12-31")
    
    # 按作业Q3规则划分数据集
    train_mask = (df['month'] >= '1957-01-01') & (df['month'] <= '1974-12-31')
    val_mask = (df['month'] >= '1975-01-01') & (df['month'] <= '1986-12-31')
    test_mask = (df['month'] >= '1987-01-01') & (df['month'] <= '2016-12-31')
    
    print(f"训练集样本数: {train_mask.sum()}")
    print(f"验证集样本数: {val_mask.sum()}")
    print(f"测试集样本数: {test_mask.sum()}")
    
    if train_mask.sum() == 0:
        # 此时如果训练集仍为0，则说明1957-1974的数据在 .parquet 文件中根本不存在
        raise ValueError("训练集样本为0。即使经过插值修正，数据仍然缺失。\n"
                         "请确认您的 features.parquet 文件 *物理上* 包含 1957-1974 年的数据。")
    
    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]
    
    # 特征标准化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # 训练基准弹性网络模型
    en_baseline = ElasticNet(
        alpha=0.5, 
        l1_ratio=0.5, 
        random_state=42,
        max_iter=10000 
    )
    en_baseline.fit(X_train_scaled, y_train)
    
    # 计算各集预测值与残差
    y_val_pred = en_baseline.predict(X_val_scaled)
    y_test_pred = en_baseline.predict(X_test_scaled)
    resid_val = y_val - y_val_pred
    resid_test = y_test - y_test_pred
    
    val_r2 = r2_oos_gkx(y_val, y_val_pred)
    test_r2 = r2_oos_gkx(y_test, y_test_pred)
    print(f"验证集 OOS R² (GKX): {val_r2:.6f}")
    print(f"测试集 OOS R² (GKX): {test_r2:.6f}")
    
    return (en_baseline, scaler, 
            X_val_scaled, X_test_scaled, 
            y_val, y_test, 
            y_val_pred, y_test_pred, 
            resid_val, resid_test, 
            df[val_mask], df[test_mask])

This function executes the first core diagnostic check: analyzing the statistical properties of the Test Set residuals (resid_test).

It calculates and prints key statistics such as the residual mean (ideally zero), standard deviation, skewness, and kurtosis.

It performs the Shapiro-Wilk test on a sample of 5,000 residuals to formally test the assumption of normality.

It calculates and reports the percentage of residuals that are considered extreme outliers based on the 1.5 times Interquartile Range (IQR) rule.

In [5]:
def analyze_residual_distribution(residuals):
    """残差分布分析（Q7核心诊断1）"""
    print("\n=== 残差分布分析 ===")
    
    mean_res = residuals.mean()
    std_res = residuals.std()
    skewness = stats.skew(residuals)
    kurtosis = stats.kurtosis(residuals) # (费雪定义, 0为正态)
    
    print(f"残差均值: {mean_res:.6f}（理想≈0）")
    print(f"残差标准差: {std_res:.6f}")
    print(f"偏度: {skewness:.4f}（理想≈0）")
    print(f"峰度 (Fisher): {kurtosis:.4f}（理想≈0）")
    
    sample_residuals = residuals.sample(min(5000, len(residuals)), random_state=42)
    shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)
    print(f"Shapiro-Wilk检验 (N=5000): 统计量={shapiro_stat:.4f}, p值={shapiro_p:.6f}（>0.05为正态）")
    
    Q1, Q3 = np.percentile(residuals, [25, 75])
    IQR = Q3 - Q1
    outliers = residuals[(residuals < Q1 - 1.5*IQR) | (residuals > Q3 + 1.5*IQR)]
    print(f"异常值占比: {len(outliers)/len(residuals)*100:.2f}%")
    
    return {
        'mean': mean_res, 'std': std_res, 'skewness': skewness, 
        'kurtosis': kurtosis, 'shapiro_p': shapiro_p, 'outlier_ratio': len(outliers)/len(residuals)
    }

This function creates a visualization summary for the residual analysis performed in the previous block.It generates a $2 \times 2$ figure containing four standard diagnostic plots:A histogram of the residuals compared to a fitted Normal distribution.A Q-Q plot to visually assess normality against the theoretical quantiles.A scatter plot of residuals versus predicted values, used to check for heteroscedasticity (non-constant variance).A box plot of the residuals.The resulting figure is saved as residual_diagnostics.png.

In [6]:
def plot_residual_diagnostics(residuals, y_test, y_test_pred):
    """绘制残差诊断图（Q7核心诊断1的可视化）"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle("Elastic Net: Residual Distribution Diagnostics (Test Set)", fontsize=14, fontweight="bold")
    
    # 1. 残差直方图 + 正态分布对比
    ax1 = axes[0, 0]
    iqr = np.percentile(residuals, 75) - np.percentile(residuals, 25)
    bin_width = 2 * iqr / (len(residuals)**(1/3))
    bins = int((residuals.max() - residuals.min()) / bin_width) if bin_width > 0 else 50
    bins = min(bins, 200) # 限制最大 bins 数量
    
    ax1.hist(residuals, bins=bins, density=True, alpha=0.7, color='skyblue', label='Residuals')
    mu, sigma = residuals.mean(), residuals.std()
    x_norm = np.linspace(mu - 4*sigma, mu + 4*sigma, 100)
    y_norm = stats.norm.pdf(x_norm, mu, sigma)
    ax1.plot(x_norm, y_norm, 'r--', linewidth=2, label=f'Normal(N={mu:.4f}, σ={sigma:.4f})')
    ax1.set_xlabel('Residuals')
    ax1.set_ylabel('Density')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(mu - 4*sigma, mu + 4*sigma) 
    
    # 2. Q-Q图
    ax2 = axes[0, 1]
    stats.probplot(residuals.sample(min(5000, len(residuals)), random_state=42), dist="norm", plot=ax2)
    ax2.set_title('Q-Q Plot (Normality Test)')
    ax2.grid(True, alpha=0.3)
    
    # 3. 残差 vs 预测值
    ax3 = axes[1, 0]
    ax3.scatter(y_test_pred, residuals, alpha=0.1, s=1) 
    ax3.axhline(y=0, color='r', linestyle='--', alpha=0.8)
    ax3.set_xlabel('Predicted Excess Returns')
    ax3.set_ylabel('Residuals')
    ax3.set_title('Residuals vs Predicted Values (Heteroscedasticity Check)')
    ax3.grid(True, alpha=0.3)
    
    # 4. 残差箱线图
    ax4 = axes[1, 1]
    box_plot = ax4.boxplot(residuals, patch_artist=True, showfliers=False) 
    box_plot['boxes'][0].set_facecolor('lightblue')
    ax4.set_ylabel('Residuals')
    ax4.set_title('Box Plot (Outlier Detection - Fliers Hidden)')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f"{RESULTS_DIR}/residual_diagnostics.png", dpi=300, bbox_inches='tight')
    plt.close()
    print(f"\n残差诊断图已保存至 {RESULTS_DIR}/residual_diagnostics.png")

This function performs the second core diagnostic, assessing whether the model's performance varies across different firm sizes.

It uses the lagged market equity feature (c_mvel1) to create five size-based quantiles (buckets) per month in the test set.

It then calculates the key performance metrics (OOS R² and residual statistics) separately for each of the five size buckets.

The results are printed, and a series of histograms comparing the residual distributions across the buckets are plotted and saved as size_bucket_analysis.png.

In [7]:
def analyze_size_buckets(df_test, residuals, y_test, y_test_pred):
    """按公司规模分组分析（Q7核心诊断2）"""
    print("\n=== 按公司规模分组分析 ===")
    
    if 'c_mvel1' in df_test.columns:
        df_test_copy = df_test.copy()
        
        residuals_series = pd.Series(residuals, index=df_test_copy.index)
        y_test_series = pd.Series(y_test, index=df_test_copy.index)
        y_test_pred_series = pd.Series(y_test_pred, index=df_test_copy.index)
        
        if not df_test_copy.index.is_unique:
             print("警告：df_test 索引不唯一，重置索引以进行规模分组。")
             df_test_copy = df_test_copy.reset_index(drop=True)
             residuals_series = residuals_series.reset_index(drop=True)
             y_test_series = y_test_series.reset_index(drop=True)
             y_test_pred_series = y_test_pred_series.reset_index(drop=True)

        df_test_copy['size_bucket'] = df_test_copy.groupby(df_test_copy['month_period'])['c_mvel1'].transform(
            lambda x: pd.qcut(x, q=5, labels=['1_Small', '2_Small-Mid', '3_Mid', '4_Mid-Large', '5_Large'], duplicates='drop')
        )
        
        size_metrics_list = []
        for bucket_name, group in df_test_copy.groupby('size_bucket'):
            group_indices = group.index
            
            oos_r2 = r2_oos_gkx(
                y_test_series[group_indices], 
                y_test_pred_series[group_indices]
            )
            
            size_metrics_list.append({
                'size_bucket': bucket_name,
                'sample_count': len(group),
                'oos_r2_gkx': oos_r2,
                'resid_mean': residuals_series[group_indices].mean(),
                'resid_std': residuals_series[group_indices].std()
            })
        
        size_metrics = pd.DataFrame(size_metrics_list).set_index('size_bucket').round(6)
        
        print("规模分组性能指标：")
        print(size_metrics.to_string())
        
        # 绘制分组残差分布对比图
        plt.figure(figsize=(15, 6))
        for i, bucket_name in enumerate(size_metrics.index):
            plt.subplot(1, 5, i+1)
            group_resid = residuals_series[df_test_copy['size_bucket'] == bucket_name]
            
            if group_resid.empty:
                continue

            plt.hist(group_resid, bins=30, alpha=0.7, density=True, color=f'C{i}')
            plt.xlabel('Residuals')
            if i == 0:
                plt.ylabel('Density')
            plt.title(f'{bucket_name}\nR²={size_metrics.loc[bucket_name, "oos_r2_gkx"]:.4f}')
            plt.grid(True, alpha=0.3)
            plt.xlim(np.percentile(residuals, 1), np.percentile(residuals, 99))
        
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/size_bucket_analysis.png", dpi=300, bbox_inches='tight')
        plt.close()
        print(f"规模分组分析图已保存至 {RESULTS_DIR}/size_bucket_analysis.png")
    else:
        warnings.warn("未找到市值特征（c_mvel1），跳过规模分组分析")

This function performs the third core diagnostic, checking for signs of overfitting by comparing the model's stability between the Validation and Test sets.

It calculates the 36-month rolling OOS R² time series for both the Validation period and the Test period.

It compares the average rolling R² from both periods. A difference greater than 0.002 (0.2%) between the Validation and Test averages is flagged as a sign of potential overfitting.

The rolling R² time series are plotted together on a single graph and saved as overfit_test.png.

In [8]:
def check_overfit(resid_val, y_val, resid_test, y_test, df_val, df_test):
    """过拟合特征检验（Q7核心诊断3：Val vs Test性能对比）"""
    print("\n=== 过拟合特征检验（Val vs Test） ===")
    
    y_val_series = pd.Series(y_val, index=df_val.index)
    y_test_series = pd.Series(y_test, index=df_test.index)
    resid_val_series = pd.Series(resid_val, index=df_val.index)
    resid_test_series = pd.Series(resid_test, index=df_test.index)

    def calculate_rolling_r2(residuals_s, y_true_s, df, window=36):
        if not residuals_s.index.equals(df.index) or not y_true_s.index.equals(df.index):
             print("警告(check_overfit): 索引不对齐，重置索引。")
             df = df.reset_index(drop=True)
             residuals_s = residuals_s.reset_index(drop=True)
             y_true_s = y_true_s.reset_index(drop=True)
             
        monthly_resid_sq_sum = residuals_s.groupby(df['month_period']).apply(lambda x: (x**2).sum())
        monthly_y_sq_sum = y_true_s.groupby(df['month_period']).apply(lambda x: (x**2).sum())
        
        monthly_agg = pd.DataFrame({
            'resid_sq_sum': monthly_resid_sq_sum,
            'y_sq_sum': monthly_y_sq_sum
        })
        
        monthly_agg['month_date'] = df.groupby(df['month_period'])['month'].first()
        
        monthly_agg['rolling_resid_sq'] = monthly_agg['resid_sq_sum'].rolling(window=window).sum()
        monthly_agg['rolling_y_sq'] = monthly_agg['y_sq_sum'].rolling(window=window).sum()
        
        monthly_agg['rolling_r2'] = 1 - (monthly_agg['rolling_resid_sq'] / monthly_agg['rolling_y_sq'].replace(0, np.nan))
        return monthly_agg.dropna(subset=['rolling_r2'])
    
    val_rolling = calculate_rolling_r2(resid_val_series, y_val_series, df_val)
    test_rolling = calculate_rolling_r2(resid_test_series, y_test_series, df_test)
    
    val_avg_r2 = val_rolling['rolling_r2'].mean()
    test_avg_r2 = test_rolling['rolling_r2'].mean()
    r2_diff = val_avg_r2 - test_avg_r2
    overfit_flag = "是" if r2_diff > 0.002 else "否" 
    
    print(f"验证集36个月滚动平均R²: {val_avg_r2:.6f}")
    print(f"测试集36个月滚动平均R²: {test_avg_r2:.6f}")
    print(f"Val - Test R²差值: {r2_diff:.6f}")
    print(f"是否过拟合: {overfit_flag}（差值>0.2%为过拟合）")
    
    plt.figure(figsize=(14, 6))
    plt.plot(val_rolling['month_date'], val_rolling['rolling_r2']*100, 
             label=f'Validation Set (1975-1986, Avg: {val_avg_r2:.4f})', color='blue', linewidth=2)
    plt.plot(test_rolling['month_date'], test_rolling['rolling_r2']*100, 
             label=f'Test Set (1987-2016, Avg: {test_avg_r2:.4f})', color='orange', linewidth=2, linestyle='--')
    plt.axhline(y=0, color='red', linestyle=':', linewidth=1.5, label='R²=0 Baseline')
    plt.xlabel('Date')
    plt.ylabel('36-Month Rolling OOS R² (GKX) [%]')
    plt.title('Overfitting Test: Val vs Test Rolling R²')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.text(0.02, 0.95, f"Overfit Check: {overfit_flag}\n(Diff = {r2_diff:.4f})",
             transform=plt.gca().transAxes, bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.8),
             verticalalignment="top")
    
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/overfit_test.png", dpi=300, bbox_inches='tight')
    plt.close()
    print(f"过拟合检验图已保存至 {RESULTS_DIR}/overfit_test.png")

This function performs the final core task of hyperparameter tuning for the Elastic Net model.

It defines a set of candidate $\alpha$ values (regularization strengths) while fixing the $l1\_ratio$ to 0.5.It trains a separate Elastic Net model for each $\alpha$ on the Training set.

Model Selection: It selects the best set of parameters based on the highest GKX OOS R² achieved on the Validation set.

It prints the performance of the optimal model on both the Validation and Test sets and saves the complete tuning results to a CSV file (elastic_net_tuning_results.csv).

In [9]:
def tuning_elastic_net(df, X, y, scaler):
    """弹性网络调优尝试（Q7核心任务4）"""
    print("\n=== 弹性网络调优尝试 ===")
    
    train_mask = (df['month'] >= '1957-01-01') & (df['month'] <= '1974-12-31')
    val_mask = (df['month'] >= '1975-01-01') & (df['month'] <= '1986-12-31')
    test_mask = (df['month'] >= '1987-01-01') & (df['month'] <= '2016-12-31')
    
    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]
    
    # 标准化
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    alpha_candidates = [0.01, 0.1, 0.5, 1.0, 5.0] 
    l1_ratio_fixed = 0.5
    tuning_results = []
    
    print(f"调优参数: l1_ratio={l1_ratio_fixed} (固定), alpha 候选: {alpha_candidates}")
    
    for alpha in alpha_candidates:
        en_tuned = ElasticNet(
            alpha=alpha,
            l1_ratio=l1_ratio_fixed,
            random_state=42,
            max_iter=10000
        )
        en_tuned.fit(X_train_scaled, y_train)
        
        y_val_pred = en_tuned.predict(X_val_scaled)
        y_test_pred = en_tuned.predict(X_test_scaled)
        
        val_r2 = r2_oos_gkx(y_val, y_val_pred)
        test_r2 = r2_oos_gkx(y_test, y_test_pred)
        test_resid_std = (y_test - y_test_pred).std()
        
        tuning_results.append({
            'alpha': alpha,
            'l1_ratio': l1_ratio_fixed,
            'val_r2_gkx': val_r2,
            'test_r2_gkx': test_r2,
            'test_resid_std': test_resid_std
        })
        
        print(f"  alpha={alpha:5.2f}: Val R²={val_r2:.6f}, Test R²={test_r2:.6f}, 残差标准差={test_resid_std:.6f}")
    
    tuning_df = pd.DataFrame(tuning_results).round(6)
    
    best_idx = tuning_df['val_r2_gkx'].idxmax()
    best_params = tuning_df.iloc[best_idx]
    
    print(f"\n最佳调优参数 (基于 Val R²): alpha={best_params['alpha']}, l1_ratio={best_params['l1_ratio']}")
    print(f"最佳参数的 *验证集* 性能: Val R²={best_params['val_r2_gkx']:.6f}")
    print(f"最佳参数的 *测试集* 性能: Test R²={best_params['test_r2_gkx']:.6f}, 残差标准差={best_params['test_resid_std']:.6f}")
    
    tuning_df.to_csv(f"{RESULTS_DIR}/elastic_net_tuning_results.csv", index=False)
    print(f"调优结果已保存至 {RESULTS_DIR}/elastic_net_tuning_results.csv")
    
    return tuning_df

In [12]:
df, X, y, feature_cols = load_data()

加载数据...
特征列数: 176
...开始月度中位数插值...
...月度中位数插值完成。
...开始后备插值 (fillna(0))...
...后备插值完成。
数据加载完成，最终有效样本数: 4320692


In [13]:
# 2. 训练基准弹性网络模型
(en_baseline, scaler, 
X_val_scaled, X_test_scaled, 
y_val, y_test, 
y_val_pred, y_test_pred, 
resid_val, resid_test, 
df_val, df_test) = train_elastic_net(df, X, y)


训练弹性网络模型（基准参数：alpha=0.5, l1_ratio=0.5）...
数据时间范围: 1957-01-31 到 2021-11-30
训练集时间范围: 1957-01-01 到 1974-12-31
验证集时间范围: 1975-01-01 到 1986-12-31
测试集时间范围: 1987-01-01 到 2016-12-31
训练集样本数: 501162
验证集样本数: 812160
测试集样本数: 2628130
验证集 OOS R² (GKX): -0.000548
测试集 OOS R² (GKX): -0.000262


In [14]:
residual_stats = analyze_residual_distribution(resid_test)
plot_residual_diagnostics(resid_test, y_test, y_test_pred)


=== 残差分布分析 ===
残差均值: 0.007542（理想≈0）
残差标准差: 0.179207
偏度: 7.4210（理想≈0）
峰度 (Fisher): 386.5944（理想≈0）
Shapiro-Wilk检验 (N=5000): 统计量=0.6654, p值=0.000000（>0.05为正态）
异常值占比: 9.70%

残差诊断图已保存至 results/q7_diagnostics/residual_diagnostics.png


In [15]:
analyze_size_buckets(df_test, resid_test, y_test, y_test_pred)


=== 按公司规模分组分析 ===
规模分组性能指标：
             sample_count  oos_r2_gkx  resid_mean  resid_std
size_bucket                                                 
1_Small            525780   -0.000198    0.012400   0.265881
2_Small-Mid        525551   -0.000191    0.005732   0.181501
3_Mid              525542   -0.000259    0.005713   0.155752
4_Mid-Large        525551   -0.000375    0.006701   0.140489
5_Large            525706   -0.000614    0.007162   0.113585
规模分组分析图已保存至 results/q7_diagnostics/size_bucket_analysis.png


In [16]:
check_overfit(resid_val, y_val, resid_test, y_test, df_val, df_test)


=== 过拟合特征检验（Val vs Test） ===
验证集36个月滚动平均R²: -0.000577
测试集36个月滚动平均R²: -0.000344
Val - Test R²差值: -0.000233
是否过拟合: 否（差值>0.2%为过拟合）
过拟合检验图已保存至 results/q7_diagnostics/overfit_test.png


In [17]:
tuning_df = tuning_elastic_net(df, X, y, scaler)


=== 弹性网络调优尝试 ===
调优参数: l1_ratio=0.5 (固定), alpha 候选: [0.01, 0.1, 0.5, 1.0, 5.0]
  alpha= 0.01: Val R²=0.002134, Test R²=-0.002638, 残差标准差=0.179539
  alpha= 0.10: Val R²=-0.000548, Test R²=-0.000262, 残差标准差=0.179207
  alpha= 0.50: Val R²=-0.000548, Test R²=-0.000262, 残差标准差=0.179207
  alpha= 1.00: Val R²=-0.000548, Test R²=-0.000262, 残差标准差=0.179207
  alpha= 5.00: Val R²=-0.000548, Test R²=-0.000262, 残差标准差=0.179207

最佳调优参数 (基于 Val R²): alpha=0.01, l1_ratio=0.5
最佳参数的 *验证集* 性能: Val R²=0.002134
最佳参数的 *测试集* 性能: Test R²=-0.002638, 残差标准差=0.179539
调优结果已保存至 results/q7_diagnostics/elastic_net_tuning_results.csv
